<h1>Plotting White Shark Data</h1>

In [ ]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import movingpandas as mpd
import contextily as ctx
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from pyproj import CRS
from keplergl import KeplerGl

# Import some helper functions from another file
%load_ext autoreload
%aimport plotting_helpers
from plotting_helpers import *

# Set matplotlib max animation size
matplotlib.rcParams['animation.embed_limit'] = 2**128

<h3>1. Load data from CSV files</h3>

In [ ]:
# Load receiver array positions
receivers_gdf = pd.read_csv('../data/VPS-Station-Locations.csv')
receivers_gdf = gpd.GeoDataFrame(receivers_gdf[['Station']], geometry=gpd.points_from_xy(receivers_gdf.Lng, receivers_gdf.Lat))
receivers_gdf = receivers_gdf.set_crs('EPSG:4326')

In [ ]:
# Compute bounds and width and height of receiver array
point = receivers_gdf['geometry'][0]
max_lon, max_lat = point.coords[0]
min_lon, min_lat = point.coords[0]
for index, row in receivers_gdf.iterrows():
    lon, lat = row['geometry'].coords[0]
    max_lon = max(max_lon, lon)
    max_lat = max(max_lat, lat)
    min_lon = min(min_lon, lon)
    min_lat = min(min_lat, lat)
bounds_gdf = gpd.GeoDataFrame([[Point(min_lon, min_lat)], [Point(max_lon, max_lat)]], geometry='geometry', crs={'init': 'epsg:4326'}, columns=['geometry'])
aeqd = CRS(proj='aeqd', ellps='WGS84', datum='WGS84', lat_0=min_lat, lon_0=min_lon).srs
bounds_gdf = bounds_gdf.to_crs(crs=aeqd)

width, height = bounds_gdf['geometry'][1].coords[0]
print('Width of receiver array is', width, 'meters and height is', height, 'meters.')

In [ ]:
# Set the filename
filename = '../data/SharkArray-2020-05-21.csv'

# Load shark positions data into a GeoDataFrame
shark_gdf = pd.read_csv(filename)
shark_gdf['t'] = pd.to_datetime(shark_gdf['DATETIME'])
shark_gdf = gpd.GeoDataFrame(shark_gdf[['TRANSMITTER', 't']], geometry=gpd.points_from_xy(shark_gdf.LON, shark_gdf.LAT))
shark_gdf = shark_gdf.set_crs('EPSG:4326')
shark_gdf = shark_gdf.set_index('t').tz_localize(None)

In [ ]:
shark_gdf['TRANSMITTER'].unique()

In [ ]:
print('The dataset contains', shark_gdf.shape[0], 'rows and', shark_gdf.shape[1], 'columns.')
print('The column names are:', list(shark_gdf.columns.values))

<h3>2. Create moving pandas trajectories from the shark position data</h3>

In [ ]:
# Create separate trajectories for each shark based on their transmitter ID
traj_collection = mpd.TrajectoryCollection(shark_gdf, 'TRANSMITTER')
print(traj_collection)

In [ ]:
# Retrieve shark 2020-20's trajectory
traj = traj_collection.filter('TRANSMITTER', ['2020-20']).trajectories[0]
print("Trajectory duration is", traj.get_duration())
print("Num points is", traj.df.shape[0])
# traj.plot(column='speed', linewidth=5, capstyle='round', figsize=(9, 5), legend=True)

In [ ]:
# Split shark trajectory by observation gap
day_trajs = mpd.TemporalSplitter(traj).split(mode='day')
print(day_trajs)

In [ ]:
# Retrieve trajectory from a single day
day_traj = day_trajs.trajectories[0]
print("Num points is", day_traj.df.shape[0])
day_traj.plot(column='speed', linewidth=5, capstyle='round', figsize=(9, 5), legend=True)
plt.title(start_to_end(day_traj))

In [ ]:
# Split the day trajectory by gaps in observation of more than 15 minutes
obs_gap_trajs = mpd.ObservationGapSplitter(day_traj).split(gap=timedelta(minutes=15))
plot_trajs(obs_gap_trajs, legend=True, rep_traj=day_traj, padding=3, figsize=(9, 9), receivers=receivers_gdf)

In [ ]:
# Split the day trajectory by stops of more than a minute in a diameter of 10 meters
stop_trajs = mpd.StopSplitter(day_traj).split(max_diameter=10, min_duration=timedelta(seconds=60))
plot_trajs(stop_trajs, title='Trajectories split by stops within 10 meters for 1 minute or more', legend=True, rep_traj=day_traj, padding=1.3, figsize=(9, 9), receivers=receivers_gdf)

<h3>3. Start animating trajectories</h3>

In [ ]:
# Animate the day trajectory
ani = animate_trajectories([day_traj], num_frames=100, interval=100, padding=1.4, rep_traj=day_traj, receivers=receivers_gdf)
HTML(ani.to_jshtml())

In [ ]:
# ani.save('./animations/shark_2020-20_day_2020_5_21_receivers.gif', fps=10)

In [ ]:
# Split the trajectories of all sharks by day
day_trajs_dict = {}

for traj in traj_collection.trajectories:
    idx = traj.df['TRANSMITTER'][0]
    day_trajs_dict[idx] = mpd.TemporalSplitter(traj).split(mode='day')

In [ ]:
# Extract the trajectories of all sharks on 1 day
day_trajs_5_21 = []
start_day = datetime(2020, 5, 21, 0, 0, 0)
delta = timedelta(days=1)

for _, d in day_trajs_dict.items():
    day_trajs_5_21 += filter_traj_col(d, start_day, start_day + delta).trajectories
day_trajs_5_21 = mpd.TrajectoryCollection(day_trajs_5_21[:10])

In [ ]:
# Animate the day trajectories
ani = animate_trajectories(day_trajs_5_21.trajectories, num_frames=100, interval=100, padding=4, rep_traj=day_traj, receivers=receivers_gdf)
HTML(ani.to_jshtml())

In [ ]:
# ani.save('./animations/all_sharks_day_2020_5_21_receivers.gif', fps=10)

<h3>4. Interactive visualization with KeplerGl</h3>

In [ ]:
# Put all trajectories in a single GeoDataFrame
data = []
for traj in day_trajs_5_21.trajectories:
    for i, (timestamp, row) in zip(range(traj.df.shape[0]), traj.df.iterrows()):
        if i == 0:
            continue
        point = row['geometry']
        prev_point = traj.df['geometry'][i - 1]
        x, y = point.coords[0]
        prev_x, prev_y = prev_point.coords[0]
        data.append([str(timestamp), x, y, prev_x, prev_y, row['TRANSMITTER'], point])
trajectories_gdf = gpd.GeoDataFrame(data, columns=['t', 'lon', 'lat', 'prev_lon', 'prev_lat', 'TRANSMITTER', 'geometry'])

In [ ]:
# Create KeplerGl instance and add data
m = KeplerGl(height=600)
m.add_data(receivers_gdf.copy(), 'receivers')
m.add_data(trajectories_gdf.copy(), 'trajectories')

In [ ]:
# Analyze trajectories
day_trajs_5_21_agg = mpd.TrajectoryCollectionAggregator(day_trajs_5_21, width * 0.1, width * 0.01, timedelta(seconds=3600))

In [ ]:
# Extract significant points from trajectory aggregation
significant_points_gdf = day_trajs_5_21_agg.get_significant_points_gdf()
significant_points_gdf['lon'] = [point.coords[0][0] for point in significant_points_gdf['geometry']]
significant_points_gdf['lat'] = [point.coords[0][1] for point in significant_points_gdf['geometry']]

In [ ]:
m.add_data(significant_points_gdf.copy(), 'significant points')
m.add_data(day_trajs_5_21_agg.get_clusters_gdf().copy(), 'clusters')
m.add_data(day_trajs_5_21_agg.get_flows_gdf().copy(), 'flows')

In [ ]:
config = { "version": "v1", "config": { "visState": { "filters": [ { "dataId": [ "trajectories" ], "id": "ohkb0fv1t", "name": [ "t" ], "type": "timeRange", "value": [ 1590019380000, 1590020996999.9995 ], "enlarged": False, "plotType": "histogram", "animationWindow": "free", "yAxis": None }, { "dataId": [ "interpolated" ], "id": "kv35ggm5", "name": [ "t" ], "type": "timeRange", "value": [ 1590019631700, 1590022037700 ], "enlarged": True, "plotType": "histogram", "animationWindow": "free", "yAxis": None } ], "layers": [ { "id": "ww1rpim", "type": "geojson", "config": { "dataId": "receivers", "label": "receivers", "color": [ 82, 151, 218 ], "columns": { "geojson": "geometry" }, "isVisible": True, "visConfig": { "opacity": 0.8, "strokeOpacity": 0.8, "thickness": 0.5, "strokeColor": None, "colorRange": { "name": "Global Warming", "type": "sequential", "category": "Uber", "colors": [ "#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300" ] }, "strokeColorRange": { "name": "Global Warming", "type": "sequential", "category": "Uber", "colors": [ "#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300" ] }, "radius": 20, "sizeRange": [ 0, 10 ], "radiusRange": [ 0, 50 ], "heightRange": [ 0, 500 ], "elevationScale": 5, "stroked": False, "filled": True, "enable3d": False, "wireframe": False }, "hidden": False, "textLabel": [ { "field": None, "color": [ 255, 255, 255 ], "size": 18, "offset": [ 0, 0 ], "anchor": "start", "alignment": "center" } ] }, "visualChannels": { "colorField": None, "colorScale": "quantile", "sizeField": None, "sizeScale": "linear", "strokeColorField": None, "strokeColorScale": "quantile", "heightField": None, "heightScale": "linear", "radiusField": None, "radiusScale": "linear" } }, { "id": "9q6dt4", "type": "point", "config": { "dataId": "interpolated", "label": "interpolated positions", "color": [ 231, 159, 213 ], "columns": { "lat": "lat", "lng": "lon", "altitude": None }, "isVisible": True, "visConfig": { "radius": 10, "fixedRadius": False, "opacity": 0.8, "outline": False, "thickness": 2, "strokeColor": None, "colorRange": { "name": "UberPool 6", "type": "diverging", "category": "Uber", "colors": [ "#213E9A", "#551EAD", "#C019BD", "#D31256", "#E6470A", "#F9E200" ] }, "strokeColorRange": { "name": "Global Warming", "type": "sequential", "category": "Uber", "colors": [ "#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300" ] }, "radiusRange": [ 0, 50 ], "filled": True }, "hidden": False, "textLabel": [ { "field": None, "color": [ 255, 255, 255 ], "size": 18, "offset": [ 0, 0 ], "anchor": "start", "alignment": "center" } ] }, "visualChannels": { "colorField": { "name": "TRANSMITTER", "type": "string" }, "colorScale": "ordinal", "strokeColorField": None, "strokeColorScale": "quantile", "sizeField": None, "sizeScale": "linear" } }, { "id": "br14m9a", "type": "line", "config": { "dataId": "interpolated", "label": "interpolated trajectories", "color": [ 221, 178, 124 ], "columns": { "lat0": "prev_lat", "lng0": "prev_lon", "lat1": "lat", "lng1": "lon" }, "isVisible": True, "visConfig": { "opacity": 0.8, "thickness": 2, "colorRange": { "name": "UberPool 6", "type": "diverging", "category": "Uber", "colors": [ "#213E9A", "#551EAD", "#C019BD", "#D31256", "#E6470A", "#F9E200" ] }, "sizeRange": [ 0, 10 ], "targetColor": None }, "hidden": False, "textLabel": [ { "field": None, "color": [ 255, 255, 255 ], "size": 18, "offset": [ 0, 0 ], "anchor": "start", "alignment": "center" } ] }, "visualChannels": { "colorField": { "name": "TRANSMITTER", "type": "string" }, "colorScale": "ordinal", "sizeField": None, "sizeScale": "linear" } }, { "id": "hi24zxj", "type": "point", "config": { "dataId": "trajectories", "label": "positions", "color": [ 77, 193, 156 ], "columns": { "lat": "lat", "lng": "lon", "altitude": None }, "isVisible": False, "visConfig": { "radius": 10, "fixedRadius": False, "opacity": 0.8, "outline": False, "thickness": 2, "strokeColor": None, "colorRange": { "name": "UberPool 6", "type": "diverging", "category": "Uber", "colors": [ "#213E9A", "#551EAD", "#C019BD", "#D31256", "#E6470A", "#F9E200" ] }, "strokeColorRange": { "name": "Global Warming", "type": "sequential", "category": "Uber", "colors": [ "#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300" ] }, "radiusRange": [ 0, 50 ], "filled": True }, "hidden": False, "textLabel": [ { "field": None, "color": [ 255, 255, 255 ], "size": 18, "offset": [ 0, 0 ], "anchor": "start", "alignment": "center" } ] }, "visualChannels": { "colorField": { "name": "TRANSMITTER", "type": "string" }, "colorScale": "ordinal", "strokeColorField": None, "strokeColorScale": "quantile", "sizeField": None, "sizeScale": "linear" } }, { "id": "xwnxp1e", "type": "line", "config": { "dataId": "trajectories", "label": "trajectories", "color": [ 18, 147, 154 ], "columns": { "lat0": "prev_lat", "lng0": "prev_lon", "lat1": "lat", "lng1": "lon" }, "isVisible": False, "visConfig": { "opacity": 0.8, "thickness": 2, "colorRange": { "name": "UberPool 6", "type": "diverging", "category": "Uber", "colors": [ "#213E9A", "#551EAD", "#C019BD", "#D31256", "#E6470A", "#F9E200" ] }, "sizeRange": [ 0, 10 ], "targetColor": None }, "hidden": False, "textLabel": [ { "field": None, "color": [ 255, 255, 255 ], "size": 18, "offset": [ 0, 0 ], "anchor": "start", "alignment": "center" } ] }, "visualChannels": { "colorField": { "name": "TRANSMITTER", "type": "string" }, "colorScale": "ordinal", "sizeField": None, "sizeScale": "linear" } }, { "id": "47vime", "type": "geojson", "config": { "dataId": "clusters", "label": "clusters", "color": [ 98, 64, 37 ], "columns": { "geojson": "geometry" }, "isVisible": False, "visConfig": { "opacity": 0.39, "strokeOpacity": 0.8, "thickness": 0.5, "strokeColor": None, "colorRange": { "name": "Global Warming", "type": "sequential", "category": "Uber", "colors": [ "#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300" ] }, "strokeColorRange": { "name": "Global Warming", "type": "sequential", "category": "Uber", "colors": [ "#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300" ] }, "radius": 10, "sizeRange": [ 0, 10 ], "radiusRange": [ 10, 50 ], "heightRange": [ 0, 500 ], "elevationScale": 5, "stroked": False, "filled": True, "enable3d": False, "wireframe": False }, "hidden": False, "textLabel": [ { "field": None, "color": [ 255, 255, 255 ], "size": 18, "offset": [ 0, 0 ], "anchor": "start", "alignment": "center" } ] }, "visualChannels": { "colorField": None, "colorScale": "quantile", "sizeField": None, "sizeScale": "linear", "strokeColorField": None, "strokeColorScale": "quantile", "heightField": None, "heightScale": "linear", "radiusField": { "name": "n", "type": "integer" }, "radiusScale": "sqrt" } }, { "id": "s1u7u4c", "type": "geojson", "config": { "dataId": "significant points", "label": "significant points", "color": [ 95, 95, 95 ], "columns": { "geojson": "geometry" }, "isVisible": False, "visConfig": { "opacity": 0.8, "strokeOpacity": 0.8, "thickness": 0.5, "strokeColor": None, "colorRange": { "name": "Global Warming", "type": "sequential", "category": "Uber", "colors": [ "#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300" ] }, "strokeColorRange": { "name": "Global Warming", "type": "sequential", "category": "Uber", "colors": [ "#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300" ] }, "radius": 5, "sizeRange": [ 0, 10 ], "radiusRange": [ 0, 50 ], "heightRange": [ 0, 500 ], "elevationScale": 5, "stroked": False, "filled": True, "enable3d": False, "wireframe": False }, "hidden": False, "textLabel": [ { "field": None, "color": [ 255, 255, 255 ], "size": 18, "offset": [ 0, 0 ], "anchor": "start", "alignment": "center" } ] }, "visualChannels": { "colorField": None, "colorScale": "quantile", "sizeField": None, "sizeScale": "linear", "strokeColorField": None, "strokeColorScale": "quantile", "heightField": None, "heightScale": "linear", "radiusField": None, "radiusScale": "linear" } }, { "id": "jtc1j9", "type": "heatmap", "config": { "dataId": "significant points", "label": "significant points heatmap", "color": [ 30, 150, 190 ], "columns": { "lat": "lat", "lng": "lon" }, "isVisible": False, "visConfig": { "opacity": 0.8, "colorRange": { "name": "Uber Viz Diverging 1.5", "type": "diverging", "category": "Uber", "colors": [ "#00939C", "#5DBABF", "#BAE1E2", "#F8C0AA", "#DD7755", "#C22E00" ] }, "radius": 10 }, "hidden": False, "textLabel": [ { "field": None, "color": [ 255, 255, 255 ], "size": 18, "offset": [ 0, 0 ], "anchor": "start", "alignment": "center" } ] }, "visualChannels": { "weightField": None, "weightScale": "linear" } }, { "id": "t1tmqux", "type": "geojson", "config": { "dataId": "flows", "label": "flows", "color": [ 23, 184, 190 ], "columns": { "geojson": "geometry" }, "isVisible": False, "visConfig": { "opacity": 0.8, "strokeOpacity": 0.5, "thickness": 2, "strokeColor": [ 136, 106, 83 ], "colorRange": { "name": "Global Warming", "type": "sequential", "category": "Uber", "colors": [ "#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300" ] }, "strokeColorRange": { "name": "Uber Viz Qualitative 1.2", "type": "qualitative", "category": "Uber", "colors": [ "#12939A", "#DDB27C", "#88572C", "#FF991F", "#F15C17", "#223F9A" ] }, "radius": 10, "sizeRange": [ 0, 2.5 ], "radiusRange": [ 0, 50 ], "heightRange": [ 0, 500 ], "elevationScale": 5, "stroked": True, "filled": True, "enable3d": False, "wireframe": False }, "hidden": False, "textLabel": [ { "field": None, "color": [ 255, 255, 255 ], "size": 18, "offset": [ 0, 0 ], "anchor": "start", "alignment": "center" } ] }, "visualChannels": { "colorField": None, "colorScale": "quantile", "sizeField": { "name": "weight", "type": "integer" }, "sizeScale": "linear", "strokeColorField": None, "strokeColorScale": "quantile", "heightField": None, "heightScale": "linear", "radiusField": None, "radiusScale": "linear" } } ], "interactionConfig": { "tooltip": { "fieldsToShow": { "receivers": [ { "name": "Station", "format": None } ], "trajectories": [ { "name": "TRANSMITTER", "format": None } ], "significant points": [], "clusters": [ { "name": "n", "format": None } ], "flows": [ { "name": "weight", "format": None } ], "interpolated": [ { "name": "t", "format": None }, { "name": "TRANSMITTER", "format": None } ] }, "compareMode": False, "compareType": "absolute", "enabled": True }, "brush": { "size": 0.3, "enabled": False }, "geocoder": { "enabled": False }, "coordinate": { "enabled": False } }, "layerBlending": "normal", "splitMaps": [], "animationConfig": { "currentTime": None, "speed": 1 } }, "mapState": { "bearing": 0, "dragRotate": False, "latitude": 34.39688367038405, "longitude": -119.54227767815335, "pitch": 0, "zoom": 13.306293407643501, "isSplit": False }, "mapStyle": { "styleType": "dark", "topLayerGroups": {}, "visibleLayerGroups": { "label": True, "road": True, "border": False, "building": True, "water": True, "land": True, "3d building": False }, "threeDBuildingColor": [ 9.665468314072013, 17.18305478057247, 31.1442867897876 ], "mapStyles": {} } } }

In [ ]:
m.save_to_html(file_name='index.html', config=config)

In [ ]:
# Create some interpolated trajectories of the day trajectories
start_time = min([traj.get_start_time() for traj in day_trajs_5_21.trajectories])
end_time = max([traj.get_end_time() for traj in day_trajs_5_21.trajectories])
num_points = 200
delta = (end_time - start_time) / num_points

data = []
for traj in day_trajs_5_21.trajectories:
    transmitter = traj.id.split('_')[0]
    point_dict = {}
    for i in range(1, num_points):
        prev_frame_time = start_time + (delta * (i - 1))
        frame_time = start_time + (delta * i)
        if traj_contains_time(traj, prev_frame_time) and traj_contains_time(traj, frame_time):
            # Retreive interpolated point
            point = traj.get_position_at(frame_time, method='interpolated')
            point_dict[frame_time] = point
            x, y = point.coords[0]
            
            # Retrieve previous interpolated point
            if prev_frame_time not in point_dict:
                prev_point = traj.get_position_at(frame_time, method='interpolated')
                point_dict[prev_frame_time] = prev_point
            else:
                prev_point = point_dict[prev_frame_time]
            prev_x, prev_y = prev_point.coords[0]
            
            # Append data
            data.append([str(frame_time), x, y, prev_x, prev_y, transmitter, point])
interpolated_gdf = gpd.GeoDataFrame(data, columns=['t', 'lon', 'lat', 'prev_lon', 'prev_lat', 'TRANSMITTER', 'geometry'])
interpolated_gdf.head()

In [ ]:
m.add_data(interpolated_gdf.copy(), 'interpolated')

In [ ]:
df = pd.DataFrame([[Point(0, 0)], [Point(1, 1)]], columns=['p'])